#### Patrick Duffy

National Renewable Energy Lab
COE Review 2020 - Fixed Bottom
Last updated: 2021/9/21

In [18]:
import pandas as pd
from ORBIT import ProjectManager, load_config
from construction_finance_param import con_fin_params

### Load and check the project configuration

In [19]:
fixed_config = load_config("COE_2020_fixed_project.yaml") 

print(f"Num turbines: {fixed_config['plant']['num_turbines']}")
print(f"Turbine: {fixed_config['turbine']}")
print(f"\nSite: {fixed_config['site']}")
print(f"\nDesign phases: {fixed_config['design_phases']}")
print(f"\nInstall phases: {list(fixed_config['install_phases'].keys())}")

Num turbines: 75
Turbine: COE_2020_8MW

Site: {'depth': 34.0, 'distance': 116, 'distance_to_landfall': 50, 'mean_windspeed': 9.03}

Design phases: ['ArraySystemDesign', 'MonopileDesign', 'ScourProtectionDesign', 'ExportSystemDesign', 'OffshoreSubstationDesign']

Install phases: ['ArrayCableInstallation', 'ExportCableInstallation', 'MonopileInstallation', 'OffshoreSubstationInstallation', 'ScourProtectionInstallation', 'TurbineInstallation']


### Run

In [20]:
project = ProjectManager(fixed_config)
d = 0.3 # includes 30% downtime factor during construction to match ORCA
project.run(availability=1-d)

### Top Level Outputs

In [21]:
print(f"Installation CapEx:  {project.installation_capex_per_kw:.2f}")
print(f"System CapEx:        {project.system_capex_per_kw:.2f}")
print(f"BOS CapEx:           {project.bos_capex_per_kw:.2f}")
print(f"Turbine CapEx:       {project.turbine_capex_per_kw:.2f}")

Installation CapEx:  564.19
System CapEx:        862.24
BOS CapEx:           1426.43
Turbine CapEx:       1300.00


In [22]:
dct = project.capex_breakdown_per_kw
dct = {k:[v] for k,v in dct.items()}
df = pd.DataFrame.from_dict(dct, orient="columns")

soft_dct = con_fin_params(bos=project.bos_capex, 
               turbine_capex=project.turbine_capex,
               orbit_install_capex=project.installation_capex,
               plant_cap=600000)

for k,v in soft_dct.items():
    if k != "soft_capex":
        df[k] = v
    else:
        df["Soft"] = v
df = df.T
df.to_csv("COE_fixed_result.csv")

In [23]:
x = project.phases["MonopileDesign"]
x.detailed_output

{'total_monopile_mass': 53482.29253004964,
 'total_monopile_cost': 160446877.59014893,
 'total_transition_piece_mass': 36308.48332393459,
 'total_transition_piece_cost': 108925449.97180377}

In [24]:
print("MP", 160446877.59/600000)
print("TP", 108925449.97/600000)

MP 267.41146265000003
TP 181.54241661666666
